# Data Analysis For Hospital Patient Safety

### Data Collection

In [2]:
# Libraries
import pandas as pd
import pingouin as pg


In [3]:
#Load Data
sismanoglio = pd.read_csv('Data/Sismanoglio.csv')
evangelismos = pd.read_csv('Data/Evagelismos.csv')
genimatas = pd.read_csv('Data/Genimatas.csv')
thriasio = pd.read_csv('Data/Thriasio.csv')
mpenakio = pd.read_csv('Data/Mpenakio.csv')

frames = [sismanoglio,evangelismos,genimatas,thriasio,mpenakio]

hospitals = pd.concat(frames)

In [4]:
#Shape of Data
print('sismanoglio has', sismanoglio.shape[0], 'rows/answers and ',sismanoglio.shape[1], 'columns/questios')
print('evangelismos has', evangelismos.shape[0], 'rows/answers and ',evangelismos.shape[1], 'columns/questios')
print('genimatas has', genimatas.shape[0], 'rows/answers and ',genimatas.shape[1], 'columns/questios')
print('thriasio has', thriasio.shape[0], 'rows/answers and ',thriasio.shape[1], 'columns/questios')
print('mpenakio has', mpenakio.shape[0], 'rows/answers and ',mpenakio.shape[1], 'columns/questios')


sismanoglio has 16 rows/answers and  60 columns/questios
evangelismos has 17 rows/answers and  60 columns/questios
genimatas has 56 rows/answers and  60 columns/questios
thriasio has 12 rows/answers and  60 columns/questios
mpenakio has 2 rows/answers and  60 columns/questios


In [5]:
print('hospitlas have', hospitals.shape[0], 'rows/answers and ',hospitals.shape[1], 'columns/questios')

hospitlas have 103 rows/answers and  61 columns/questios


### Data cleaning

In [6]:
#Drop irrelevant coumns
hospitals.drop("Χρονική σήμανση", axis=1, inplace=True)

In [7]:
#print all questions
for index, item in enumerate(hospitals.columns, start=0):  
    print(index," ", item)

0   Σε ποιο Εργασιακό Περιβάλλον /Τομέα/ Μονάδα εργάζεστε;
1   Σε ποιο τμήμα του Παθολογικού Τομέα εργάζεστε;
2   Σε ποιο τμήμα του Χειρουργικού Τομέα εργάζεστε;
3   Σε ποιο τμήμα του Εργαστηριακού Τομέα εργάζεστε;
4   Παρακαλώ γράψτε το όνομα του Τομέα που εργάζεστε.
5   1. Οι εργαζόμενοι υποστηρίζουν ο ένας τον άλλον.
6   2. Υπάρχει επαρκές προσωπικό για τη διεκπεραίωση της εργασίας.
7   3. Όταν υπάρχει μεγάλος όγκος εργασίας που πρέπει να γίνει άμεσα, το προσωπικό εργάζεται ομαδικά για την ολοκλήρωση της.
8   4. Υπάρχει σεβασμός ανάμεσα στους εργαζομένους.
9   5. Το προσωπικό εργάζεται περισσότερες ώρες από αυτές που διασφαλίζουν ασφαλή φροντίδα για τον ασθενή.
10   6. Το προσωπικό εργάζεται δυναμικά για τη βελτίωση της ασφάλειας του ασθενούς.
11   7. Αξιοποιείται περισσότερο προσωρινό προσωπικό από αυτό που διασφαλίζει την ασφαλή φροντίδα του ασθενούς.
12   8. Το προσωπικό αισθάνεται ότι τα λάθη του μπορεί να χρησιμοποιηθούν εναντίον του. 
13   9. Τα λάθη οδήγησαν σε θετικές αλλαγέ

In [8]:
hospitals_jobField  = hospitals.iloc[: , :5]
hospitals_personalInfo = hospitals.iloc[:, 49:58]
hospitals_questions = hospitals.iloc[:, 5:49]

In [9]:
#print all questions
for index, item in enumerate(hospitals_questions, start=0):  
    print(index," ", item)

0   1. Οι εργαζόμενοι υποστηρίζουν ο ένας τον άλλον.
1   2. Υπάρχει επαρκές προσωπικό για τη διεκπεραίωση της εργασίας.
2   3. Όταν υπάρχει μεγάλος όγκος εργασίας που πρέπει να γίνει άμεσα, το προσωπικό εργάζεται ομαδικά για την ολοκλήρωση της.
3   4. Υπάρχει σεβασμός ανάμεσα στους εργαζομένους.
4   5. Το προσωπικό εργάζεται περισσότερες ώρες από αυτές που διασφαλίζουν ασφαλή φροντίδα για τον ασθενή.
5   6. Το προσωπικό εργάζεται δυναμικά για τη βελτίωση της ασφάλειας του ασθενούς.
6   7. Αξιοποιείται περισσότερο προσωρινό προσωπικό από αυτό που διασφαλίζει την ασφαλή φροντίδα του ασθενούς.
7   8. Το προσωπικό αισθάνεται ότι τα λάθη του μπορεί να χρησιμοποιηθούν εναντίον του. 
8   9. Τα λάθη οδήγησαν σε θετικές αλλαγές.
9   10. Από τύχη δεν συμβαίνουν σοβαρά λάθη.
10   11. Όταν υπάρχει μεγάλος φόρτος εργασίας παρέχεται βοήθεια από άλλες Μονάδες.
11   12. Όταν δηλώνεται ένα δυσμενές συμβάν, υπάρχει η αίσθηση ότι καταγράφεται το άτομο και όχι το καθαυτό συμβάν.
12   13. Όταν γίνονται αλλ

### Split Questions to 12 categories according to HSOPSC

In [10]:
#helper functions

def my_reverse(table, q):
    for index in range(len(table)):
        if table[q].iloc[index] == 1:
            table[q].iloc[index] = 5
        elif table[q].iloc[index] == 2:
            table[q].iloc[index] = 4
        elif table[q].iloc[index] == 4:
            table[q].iloc[index] = 2
        elif table[q].iloc[index] == 5:
            table[q].iloc[index] = 1
    return table

#### Parameter of patient safety 1 : Open Communication

In [11]:
g4 = '4. Το προσωπικό μπορεί ελεύθερα να αμφισβητήσει τις αποφάσεις ή τις ενέργειες των ανωτέρων του.'
g2 = '2. Το προσωπικό μιλάει ελεύθερα όταν διαπιστώσει κάτι που επιδρά αρνητικά στη φροντίδα υγείας του ασθενούς.'
g6 = '6. Το προσωπικό φοβάται να ρωτήσει, όταν κάτι δεν φαίνεται σωστό.'

open_communication = hospitals_questions[:][[g4,g2,g6]]

In [12]:
#reverse question g6
my_reverse(open_communication, g6)

,4. Το προσωπικό μπορεί ελεύθερα να αμφισβητήσει τις αποφάσεις ή τις ενέργειες των ανωτέρων του.,2. Το προσωπικό μιλάει ελεύθερα όταν διαπιστώσει κάτι που επιδρά αρνητικά στη φροντίδα υγείας του ασθενούς.,"6. Το προσωπικό φοβάται να ρωτήσει, όταν κάτι δεν φαίνεται σωστό."
0,2,1,4
1,2,1,4
2,3,4,2
3,4,4,4
4,3,5,5
...,...,...,...
9,3,4,4
10,3,4,5
11,1,3,3
0,2,3,4


In [13]:
#calculate cronabachs alpha
ca = pg.cronbach_alpha(data=open_communication)
print("Cronabachs alpha for parameter 1: ",ca[0])

Cronabachs alpha for parameter 1:  0.7009313356857754


#### Parameter of patient safety 2 


In [14]:
g1 = '1. Γίνεται συστηματική αξιολόγηση των αλλαγών που τίθενται σε εφαρμογή, βάσει δυσμενών συμβάντων που έχουν αναφερθεί. '
g3 = '3. Το προσωπικό ενημερώνεται συχνά για τα δυσμενή συμβάντα που λαμβάνουν χώρα.'
g5 = '5. Το προσωπικό συζητά τρόπους πρόληψης δυσμενών συμβάντων ώστε να μην επαναληφθούν.'


parameter_2 = hospitals_questions[:][[g1,g3,g5]]

In [15]:
#calculate cronabachs alpha
ca = pg.cronbach_alpha(data=parameter_2)
print("Cronabachs alpha for parameter 1: ",ca[0])

Cronabachs alpha for parameter 1:  0.7054077126445635


#### Parameter of patient safety 3

In [16]:
a6 = '6. Το προσωπικό εργάζεται δυναμικά για τη βελτίωση της ασφάλειας του ασθενούς.'
a9 = '9. Τα λάθη οδήγησαν σε θετικές αλλαγές.'
a13 = '13. Όταν γίνονται αλλαγές για τη βελτίωση της ασφάλειας του ασθενούς, αξιολογείται η αποτελεσματικότητά τους.'

parameter_3 = hospitals_questions[:][[a6,a9,a13]]

In [17]:
#calculate cronabachs alpha
ca = pg.cronbach_alpha(data=parameter_3)
print("Cronabachs alpha for parameter 1: ",ca[0])

Cronabachs alpha for parameter 1:  0.5784065608025519


#### Parameter of patient safety 4

In [18]:
a1 = '1. Οι εργαζόμενοι υποστηρίζουν ο ένας τον άλλον.'
a3 = '3. Όταν υπάρχει μεγάλος όγκος εργασίας που πρέπει να γίνει άμεσα, το προσωπικό εργάζεται ομαδικά για την ολοκλήρωση της.'
a4 = '4. Υπάρχει σεβασμός ανάμεσα στους εργαζομένους.'
a11 = '11. Όταν υπάρχει μεγάλος φόρτος εργασίας παρέχεται βοήθεια από άλλες Μονάδες.'

parameter_4 = hospitals_questions[:][[a1,a3,a4,a11]]

In [19]:
#calculate cronabachs alpha
ca = pg.cronbach_alpha(data=parameter_4)
print("Cronabachs alpha for parameter 1: ",ca[0])

Cronabachs alpha for parameter 1:  0.7165631888715147


#### Parameter of patient safety 5

In [20]:
a8 = '8. Το προσωπικό αισθάνεται ότι τα λάθη του μπορεί να χρησιμοποιηθούν εναντίον του. '
a12 = '12. Όταν δηλώνεται ένα δυσμενές συμβάν, υπάρχει η αίσθηση ότι καταγράφεται το άτομο και όχι το καθαυτό συμβάν.'
a16 = '16. Το προσωπικό ανησυχεί ότι τα λάθη του καταγράφονται στο προσωπικό του φάκελο.'


parameter_5 = hospitals_questions[:][[a8,a12,a16]]

In [21]:
parameter_5

,8. Το προσωπικό αισθάνεται ότι τα λάθη του μπορεί να χρησιμοποιηθούν εναντίον του.,"12. Όταν δηλώνεται ένα δυσμενές συμβάν, υπάρχει η αίσθηση ότι καταγράφεται το άτομο και όχι το καθαυτό συμβάν.",16. Το προσωπικό ανησυχεί ότι τα λάθη του καταγράφονται στο προσωπικό του φάκελο.
0,3,3.0,3
1,3,3.0,2
2,4,2.0,2
3,2,3.0,3
4,4,2.0,4
...,...,...,...
9,4,3.0,4
10,4,4.0,3
11,5,5.0,5
0,2,3.0,2


In [22]:
#reverse question a8 a12 a16
#pd.options.mode.chained_assignment = None  # default='warn'
#my_reverse(parameter_5, a12)
#my_reverse(parameter_5, a12)
#my_reverse(parameter_5, a16)

In [23]:
#calculate cronabachs alpha
ca = pg.cronbach_alpha(data=parameter_5)
print("Cronabachs alpha for parameter 1: ",ca[0])

Cronabachs alpha for parameter 1:  0.6166206065600939


#### Parameter of patient safety 6

In [24]:
b1 = '1. Επιβραβεύει το προσωπικό με καλά λόγια όταν τηρούνται οι διαδικασίες ασφαλούς φροντίδας του ασθενούς.'
b2 = '2. Λαμβάνει σοβαρά υπόψη του τις προτάσεις του προσωπικού για βελτίωση της ασφάλειας του ασθενούς.'
b3 = '3. Παροτρύνει το προσωπικό να εργάζεται γρηγορότερα όταν υπάρχει υπερβολικός φόρτος εργασίας και πίεση, ακόμη και όταν παραβλέπονται σημαντικές διαδικασίες ασφάλειας του ασθενούς.'
b4 = '4. Παραβλέπει επαναλαμβανόμενα προβλήματα ασφάλειας του ασθενούς.'

parameter_6 = hospitals_questions[:][[b1,b2,b3,b4]]



In [25]:
#reverse question b3 b4
parameter_6 = my_reverse(parameter_6, b3)
parameter_6 = my_reverse(parameter_6, b4)



/Users/stathis/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [26]:
#calculate cronabachs alpha
ca = pg.cronbach_alpha(data=parameter_6)
print("Cronabachs alpha for parameter 1: ",ca[0])

Cronabachs alpha for parameter 1:  0.7798695280142379


#### Parameter of patient safety 7

In [27]:
a2 = '2. Υπάρχει επαρκές προσωπικό για τη διεκπεραίωση της εργασίας.'
a5 = '5. Το προσωπικό εργάζεται περισσότερες ώρες από αυτές που διασφαλίζουν ασφαλή φροντίδα για τον ασθενή.'
a7 = '7. Αξιοποιείται περισσότερο προσωρινό προσωπικό από αυτό που διασφαλίζει την ασφαλή φροντίδα του ασθενούς.'
a14 = '14. Το προσωπικό εργάζεται σε «κατάσταση κρίσης» προσπαθώντας να κάνει πολλά πράγματα γρήγορα.'

parameter_7 = hospitals_questions[:][[a2,a5,a7,a14]]
parameter_7.head()

,2. Υπάρχει επαρκές προσωπικό για τη διεκπεραίωση της εργασίας.,5. Το προσωπικό εργάζεται περισσότερες ώρες από αυτές που διασφαλίζουν ασφαλή φροντίδα για τον ασθενή.,7. Αξιοποιείται περισσότερο προσωρινό προσωπικό από αυτό που διασφαλίζει την ασφαλή φροντίδα του ασθενούς.,14. Το προσωπικό εργάζεται σε «κατάσταση κρίσης» προσπαθώντας να κάνει πολλά πράγματα γρήγορα.
0,3,3.0,3,3
1,2,3.0,2,2
2,3,4.0,4,4
3,2,3.0,3,3
4,2,3.0,1,5


In [28]:
#reverse question a5 a7 a14
parameter_7 = my_reverse(parameter_7,a5)
parameter_7 = my_reverse(parameter_7,a7)
parameter_7 = my_reverse(parameter_7,a14)




/Users/stathis/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [29]:
#calculate cronabachs alpha
ca = pg.cronbach_alpha(data=parameter_7)
print("Cronabachs alpha for parameter 1: ",ca[0])

Cronabachs alpha for parameter 1:  0.5449498614191258


#### Parameter of patient safety 8

In [30]:
st1 = '1. Η Διοίκηση εξασφαλίζει εργασιακό κλίμα που προάγει την ασφάλεια του ασθενούς .'
st8 = '8. Η ασφάλεια του ασθενούς αποτελεί κύρια προτεραιότητα για την Διοίκηση.'
st9 = '9. Η Διοίκηση δείχνει να ενδιαφέρεται για την ασφάλεια του ασθενούς μόνο μετά από την αναφορά κάποιου δυσμενούς συμβάντος.'

parameter_8 = hospitals_questions[:][[st1,st8,st9]]

In [31]:
#reverse question st9
parameter_8 = my_reverse(parameter_8,st9)




In [32]:
#calculate cronabachs alpha
ca = pg.cronbach_alpha(data=parameter_8)
print("Cronabachs alpha for parameter 1: ",ca[0])

Cronabachs alpha for parameter 1:  0.7144048962711684


#### Eleftheroi paramteroi

In [33]:
z = 'Τους τελευταίους 12 μήνες πόσα δυσμενή συμβάντα έχετε αναφέρει; (Αναφέρετε έναν αριθμό)'
parameter_14 = hospitals_questions[:][[z]]

In [61]:
import numpy as np


pd.set_option('display.max_rows', None)
parameter_14 = parameter_14.dropna()

print(parameter_14)

#for i in range(len(parameter_14)):
 #   print(parameter_14.loc[i,z])





   Τους τελευταίους 12 μήνες πόσα δυσμενή συμβάντα έχετε αναφέρει; (Αναφέρετε έναν αριθμό)
0                                                   4                                     
2                                                  10                                     
4                                                   2                                     
6                                                   6                                     
12                                                  2                                     
13                                       ΟΠΩΣΔΗΠΟΤΕ 2                                     
14                                                  2                                     
15                                                  3                                     
0                                                  10                                     
1                                              ΚΑΝΕΝΑ                                     

In [35]:
e = 'Παρακαλώ αξιολογήστε το επίπεδο ασφάλειας των ασθενών συνολικά στο Εργασιακό Περιβάλλον/Μονάδα σας επιλέγοντας με √ την απάντηση που σας εκφράζει.'

parameter_13 = hospitals_questions[:][[e]]

parameter_13 = parameter_13.dropna()
#print(parameter_13.shape)

#print(parameter_13)



In [65]:
for index, item in enumerate(parameter_13, start=0):  
    print(index," ", item)

0   Παρακαλώ αξιολογήστε το επίπεδο ασφάλειας των ασθενών συνολικά στο Εργασιακό Περιβάλλον/Μονάδα σας επιλέγοντας με √ την απάντηση που σας εκφράζει.
